In [1]:
from dask_jobqueue import SLURMCluster
from dask.distributed import Client
# This will use 4 independent processes, each of which launches 8 threads, using 32 physical cores in the cluster.
# Given the total number of cores (here, 32), experiment with different number of processes to get optimal performance.
cluster = SLURMCluster(cores=32, memory='740 GiB', processes=4, interface="ib0", walltime='00:30:00')
cluster.scale(1)
client = Client(cluster)

# Using dask dashboard

1. In your laptop, do port forwarding by `ssh -N -f -L localhost:8787:localhost:8787 your_netid@stellar-vis1.princeton.edu`
2. Open a web browser: `http://127.0.0.1:8787`

In [2]:
client

Connection method: Cluster object,Cluster type: dask_jobqueue.SLURMCluster
Dashboard: http://10.33.34.34:8787/status,
Dashboard: http://10.33.34.34:8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.33.34.34:46873,Workers: 0
Dashboard: http://10.33.34.34:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


# Lazily loading the dataset

Specifying *chunksize* through optional argument `chunks=dict(x=128, y=128, z=128)` will load data as dask array.

In [3]:
import pyathena as pa
s = pa.LoadSim("/projects2/EOSTRIKE/sanghyuk/cores/M10.J4.P2.N1024")
ds = s.load_hdf5(100, chunks=dict(x=128, y=128, z=128))

Note that all the data variables are lazily loaded into memory

In [4]:
ds

<xarray.Dataset> Size: 30GB
Dimensions:     (z: 1024, y: 1024, x: 1024)
Coordinates:
  * x           (x) float64 8kB -1.998 -1.994 -1.99 -1.986 ... 1.99 1.994 1.998
  * y           (y) float64 8kB -1.998 -1.994 -1.99 -1.986 ... 1.99 1.994 1.998
  * z           (z) float64 8kB -1.998 -1.994 -1.99 -1.986 ... 1.99 1.994 1.998
Data variables:
    dens        (z, y, x) float32 4GB dask.array<chunksize=(128, 128, 128), meta=np.ndarray>
    mom1        (z, y, x) float32 4GB dask.array<chunksize=(128, 128, 128), meta=np.ndarray>
    mom2        (z, y, x) float32 4GB dask.array<chunksize=(128, 128, 128), meta=np.ndarray>
    mom3        (z, y, x) float32 4GB dask.array<chunksize=(128, 128, 128), meta=np.ndarray>
    phi         (z, y, x) float32 4GB dask.array<chunksize=(128, 128, 128), meta=np.ndarray>
    defect-phi  (z, y, x) float32 4GB dask.array<chunksize=(128, 128, 128), meta=np.ndarray>
    phigas      (z, y, x) float32 4GB dask.array<chunksize=(128, 128, 128), meta=np.ndarray>
Attributes: (12/16)
    Coordinates:    b'cartesian'
    DatasetNames:   [b'cons']
    MaxLevel:       0
    MeshBlockSize:  [64 64 64]
    NumCycles:      39439
    NumMeshBlocks:  4096
    ...             ...
    RootGridX3:     [-2.  2.  1.]
    Time:           0.3000057
    VariableNames:  [b'dens' b'mom1' b'mom2' b'mom3' b'phi' b'defect-phi' b'p...
    dx1:            0.00390625
    dx2:            0.00390625
    dx3:            0.00390625

In [5]:
ds.dens

<xarray.DataArray 'dens' (z: 1024, y: 1024, x: 1024)> Size: 4GB
dask.array<rechunk-merge, shape=(1024, 1024, 1024), dtype=float32, chunksize=(128, 128, 128), chunktype=numpy.ndarray>
Coordinates:
  * x        (x) float64 8kB -1.998 -1.994 -1.99 -1.986 ... 1.99 1.994 1.998
  * y        (y) float64 8kB -1.998 -1.994 -1.99 -1.986 ... 1.99 1.994 1.998
  * z        (z) float64 8kB -1.998 -1.994 -1.99 -1.986 ... 1.99 1.994 1.998

In [6]:
%%time
squared_sum = (ds.dens**2).sum()

CPU times: user 18.2 ms, sys: 1.08 ms, total: 19.3 ms
Wall time: 19 ms


Calculation is not performed yet!

In [7]:
%%time
squared_sum = squared_sum.compute()


CPU times: user 4.17 s, sys: 176 ms, total: 4.35 s
Wall time: 5.91 s


Actual calculation is performed only when `compute()` is explicitly called.

In [8]:
squared_sum

<xarray.DataArray 'dens' ()> Size: 4B
array(4.3192967e+10, dtype=float32)